Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [2]:
gender = df['gender'].value_counts()
count = df['gender'].shape[0]
(
(gender.index[0], round((gender[0] / count) * 100, 2)),
(gender.index[1], round((gender[1] / count) * 100, 2))
)

(('Male', 50.48), ('Female', 49.52))

##### 2. Какое количество уникальных значений у поля InternetService?

In [3]:
df['InternetService'].nunique()

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [4]:
df['TotalCharges'].dtype

dtype('O')

In [5]:
TotalCharges = df.loc[df['TotalCharges'] != ' ',
                      'TotalCharges'].astype('float32')

median_1 = TotalCharges.median()
mean_1 = TotalCharges.mean()
std_1 = TotalCharges.std()

stat_1 = list(map(lambda x: round(x, 2), [median_1, mean_1, std_1]))
stat_1

[1397.48, 2283.3, 2266.77]

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [6]:
df['PhoneService'].unique()

array(['No', 'Yes'], dtype=object)

In [7]:
df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = 1
df.loc[df['PhoneService'] == 'No', 'PhoneService'] = 0
df['PhoneService'] = df['PhoneService'].astype(int)
df['PhoneService'].unique()

array([0, 1])

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [8]:
df.loc[df['TotalCharges'] == ' ', 'TotalCharges'] = np.nan
df['TotalCharges'].fillna(0, inplace=True)
df['TotalCharges'] = df['TotalCharges'].astype('float32')

median_2 = df['TotalCharges'].median()
mean_2 = df['TotalCharges'].mean()
std_2 = df['TotalCharges'].std()

stat_2 = list(map(lambda x: round(x, 2), [median_2, mean_2, std_2]))
df['TotalCharges'].isna().sum()

0

In [9]:
display(stat_1, stat_2)

[1397.48, 2283.3, 2266.77]

[1394.55, 2279.73, 2266.79]

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [10]:
df.loc[df['Churn'] == 'Yes', 'Churn'] = 1
df.loc[df['Churn'] == 'No', 'Churn'] = 0

In [11]:
df['Churn'] = df['Churn'].astype(int)

In [12]:
df['Churn'].unique()

array([0, 1])

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [13]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].value_counts()

StreamingMovies      StreamingTV          TechSupport        
No internet service  No internet service  No internet service    1526
No                   No                   No                     1476
Yes                  Yes                  No                      996
                                          Yes                     944
No                   No                   Yes                     542
Yes                  No                   No                      508
No                   Yes                  No                      493
Yes                  No                   Yes                     284
No                   Yes                  Yes                     274
dtype: int64

In [14]:
Features = ['StreamingMovies', 'StreamingTV', 'TechSupport']

for featuer in Features:
    df.loc[df[featuer] == 'Yes', featuer] = 1
    df.loc[(df[featuer] == 'No') | (df[featuer] == 'No internet service'), 
           featuer] = 0
    
df[['StreamingMovies', 'StreamingTV', 'TechSupport']] = \
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].astype(int)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [15]:
# Пропусков нет
df['PhoneService'].isna().sum()

0

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [16]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [17]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

data = df[features]
target =df[target]

train_data, test_data, train_target, test_target = \
train_test_split(data, target, test_size=.25, random_state=1)


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [20]:
gender.fit_transform(train_data, train_target)

,gender_Female,gender_Male
6463,0,1
289,0,1
4356,1,0
6222,0,1
1790,1,0
...,...,...
905,0,1
5192,0,1
3980,0,1
235,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [21]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [22]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])


Объединение всех "кубиков" очень легко сделать таким образом

In [23]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [24]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(train_data, train_target)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [26]:
preds = pipeline.predict_proba(test_data)
preds

array([[0.96, 0.04],
       [0.4 , 0.6 ],
       [0.98, 0.02],
       ...,
       [1.  , 0.  ],
       [0.22, 0.78],
       [0.7 , 0.3 ]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [27]:
from sklearn.metrics import roc_auc_score, log_loss

log_loss(test_target, preds)

0.8583319387592321

In [28]:
roc_auc_score(test_target, pipeline.predict(test_data))

0.6654254598744961

### Сохраним наш пайплайн

In [29]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)